# Import relevant libraries

In [10]:
import os
import gdal
import glob

# Translate Lat and Lon files
- Find out information about the lat lon file
- Using the gdal_translate tool translate the lat and lon hdr files into vrt format

In [2]:
latlon_info = gdal.Info(r'D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf')
print(latlon_info)

Driver: HDF4/Hierarchical Data Format Release 4
Files: D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf
Size is 512, 512
Coordinate System is `'
Metadata:
  HDFEOSVersion=HDFEOS_V2.12
Subdatasets:
  SUBDATASET_1_NAME=HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf":latlon:lat
  SUBDATASET_1_DESC=[1200x1200] lat latlon (32-bit floating-point)
  SUBDATASET_2_NAME=HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf":latlon:lon
  SUBDATASET_2_DESC=[1200x1200] lon latlon (32-bit floating-point)
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,  512.0)
Upper Right (  512.0,    0.0)
Lower Right (  512.0,  512.0)
Center      (  256.0,  256.0)



In [3]:
gdal.Translate("h00v01_lat.vrt", r'HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf":latlon:lat', format = 'VRT' )

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000008247F81540> >

In [4]:
gdal.Translate("h00v01_lon.vrt", r'HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\MAIACLatlon.h00v01.hdf":latlon:lon', format = 'VRT' )

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000008247F81480> >

# Warp the AOT images
- get a list of names of HDF files
- for each file name in list write data.vrt (change file name in data.vrt for each of the files)
    - use os.system or os.path to write a 'data.vrt' text file in xml language 
    - open("data.vrt")
- run gdal_warp and rename projected*.tif

# Warping the image

Working now! (It was a problem with how Robin set things up!)

In [1]:
import os
os.system('gdalwarp geoloc.vrt test.tif -geoloc -s_srs "EPSG:4326" -t_srs "EPSG:27700"')

0

Working!

In [11]:
gdal.Warp('test3.tif', 'geoloc.vrt', geoloc = True, srcSRS = 'EPSG:4326', dstSRS = 'EPSG:27700')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000099E9D6AB70> >

# Get a list of files

In [8]:
with open(r'D:\Annies_Dissertation\MAIAC_Test\AOT_filenames.txt', 'w') as myfile:
    for filename in sorted(glob.glob(r'D:\Annies_Dissertation\MAIAC_Test\2014\MAIACAAOT*.hdf')):
        myfile.write(os.path.basename(filename)+ '\n')

print(myfile.closed)

True


Old code

In [5]:
import glob
myfile = open('list.txt', 'w')
for filename in sorted(glob.glob(r'D:\Annies_Dissertation\MAIAC_Test\2014\MAIACAAOT*.hdf')):
    myfile.write(os.path.basename(filename)+ '\n') 
print(myfile.closed)
myfile.close()

False


In [7]:
with open('list.txt', 'w') as myfile, open('blah.txt', 'w') as robinfile:
    myfile.write('blah')
    robinfile.write('test')
    
print(myfile.closed)

True


# Create data.vrt/ geoloc.vrt

In [30]:
open("geoloc.vrt", 'r')

<_io.TextIOWrapper name='geoloc.vrt' mode='r' encoding='cp1252'>

# Stuff that i've done and works... but improved...

In [30]:
xml = """<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"2014\MAIACAAOT.h00v01.20140011255.hdf":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>
"""

In [31]:
print(xml)

<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"2014\MAIACAAOT.h00v01.20140011255.hdf":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>



In [3]:
xml = """<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"2014\{filename}":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>
"""

In [22]:
with open(r'D:\Annies_Dissertation\MAIAC_Test\test_geoloc_2.vrt', 'w') as myfile:
    for name in sorted(glob.glob(r'D:\Annies_Dissertation\MAIAC_Test\2014\MAIACAAOT*.hdf')):
        myfile.write(xml.format(filename=(os.path.basename(name))))

print(myfile.closed)

True


files = glob()
for filename in files:
    write the VRT file
    Run GDAL warp

In [42]:
def sayhello(name, comment=None):
    print("Hello {name}".format(name=name))
    if comment is not None:
        print(comment)

In [2]:
sayhello('Annie')

NameError: name 'sayhello' is not defined

In [ ]:
def list_of_files(filename):
    

# Stuff that i've tried/ doesn't work and needs deleting at somepoint...

In [22]:
s = "This is a bit of text, and this is a number {filename}!"

In [25]:
b = 'rob "hello" in'

In [26]:
print(s.format(filename=b))

This is a bit of text, and this is a number rob "hello" in!


In [28]:
s = """hello
Annie"""

In [29]:
print(s)

hello
Annie


In [3]:
geoloc = open("geoloc.vrt", 'w')
geoloc.write( <VRTDataset rasterXSize="1200" rasterYSize="1200">)
geoloc.write(  <Metadata domain="GEOLOCATION">)
geoloc.write(     <mdi key="X_DATASET">lon.vrt</mdi>)
geoloc.write(     <mdi key="X_BAND">1</mdi>)
geoloc.write(     <mdi key="Y_DATASET">lat.vrt</mdi>)
geoloc.write(     <mdi key="Y_BAND">1</mdi>)
geoloc.write(     <mdi key="PIXEL_OFFSET">0</mdi>)
geoloc.write(     <mdi key="LINE_OFFSET">0</mdi>)
geoloc.write(     <mdi key="PIXEL_STEP">1</mdi>)
geoloc.write(     <mdi key="LINE_STEP">1</mdi>)
geoloc.write(  </Metadata>)
geoloc.write(  <VRTRasterBand dataType="Float32" band="1">)
geoloc.write(    <ColorInterp>Gray</ColorInterp>)
geoloc.write(    <SimpleSource>)
geoloc.write(      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"MAIACAAOT.h00v01.20131861320.hdf":grid1km:Optical_Depth_Land</SourceFilename>)
geoloc.write(      <SourceBand>1</SourceBand>)
geoloc.write(      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />)
geoloc.write(      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />)
geoloc.write(      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />)
geoloc.write(    </SimpleSource>)
geoloc.write(   </VRTRasterBand>)
geoloc.write( </VRTDataset>)

geoloc.close()

SyntaxError: invalid syntax (<ipython-input-3-d8cb80ef6175>, line 2)

In [8]:
with open(r"D:\Annies_Dissertation\MAIAC_Test\test_geoloc.vrt", 'w') as myfile:
    myfile.write(xml.format(filename='MAIACAAOT.h00v01.20140011255.hdf'))
print(myfile.closed)

True
